In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import warnings
import pandas as pd

warnings.filterwarnings('ignore')

# Load the data and format it as a dataframe
cancer = load_breast_cancer(as_frame=True)

# print the first 5 rows of the data
display(cancer.data.head())

# training model
def model_training(model, train_data: pd.DataFrame, train_target: pd.DataFrame, test_data: pd.DataFrame, test_target: pd.DataFrame):
    model.fit(train_data, train_target)
    return model


# print the accuracy of the model
def model_evaluation(model, test_data: pd.DataFrame, predict_target: pd.DataFrame):
    print(f'Model Evaluation:{ model.score(test_data,predict_target)}\n')


# print classification report
def print_classification_report(model, test_data: pd.DataFrame, test_target: pd.DataFrame):
    predict_target = model.predict(test_data)
    print(f'Classification Report:\n{classification_report(y_true=test_target, y_pred=predict_target)}\n')


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [2]:

# split the data into training and testing
cancer_data_train,cancer_data_test,cancer_target_train,cancer_target_test = train_test_split(cancer.data,cancer.target, test_size=0.2, random_state=42)

# single model list
models = [SVC(),DecisionTreeClassifier(),LogisticRegression()]


# single model training
for model in models:
    print(f'======== { model } Model Training ========')
    model_training(model=model, train_data=cancer_data_train, train_target=cancer_target_train, test_data=cancer_data_test, test_target=cancer_target_test)
    model_evaluation(model=model, test_data=cancer_data_test, predict_target=cancer_target_test)
    print_classification_report(model=model, test_data=cancer_data_test, test_target=cancer_target_test)
    print('====================================================================================================')
    

======== SVC() Model Training ========
Model Evaluation:0.9473684210526315

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.86      0.92        43
           1       0.92      1.00      0.96        71

    accuracy                           0.95       114
   macro avg       0.96      0.93      0.94       114
weighted avg       0.95      0.95      0.95       114


======== DecisionTreeClassifier() Model Training ========
Model Evaluation:0.9473684210526315

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.93      0.93        43
           1       0.96      0.96      0.96        71

    accuracy                           0.95       114
   macro avg       0.94      0.94      0.94       114
weighted avg       0.95      0.95      0.95       114


======== LogisticRegression() Model Training ========
Model Evaluation:0.956140350877193

Classification Report:
              

In [3]:
# voting classifier, voting='hard' means majority voting
voting_model = VotingClassifier(estimators=[('svm',SVC()),('dt',DecisionTreeClassifier()),('lr',LogisticRegression())],voting='hard')

# stacking classifier, final_estimator is the final estimator which will be used to predict the output
stack_model_dst = StackingClassifier(estimators=[('svm',SVC()),('ada',DecisionTreeClassifier()),('lr',LogisticRegression())],final_estimator=DecisionTreeClassifier())

# stacking classifier, final_estimator is the final estimator which will be used to predict the output
stack_model_lr = StackingClassifier(estimators=[('svm',SVC()),('ada',DecisionTreeClassifier()),('lr',LogisticRegression())],final_estimator=LogisticRegression())


ensemble_models = [voting_model,stack_model_dst, stack_model_lr]
# ensemble model training
for model in ensemble_models:
    print(f'======== { model } Model Training ========')
    model_training(model=model, train_data=cancer_data_train, train_target=cancer_target_train, test_data=cancer_data_test, test_target=cancer_target_test)
    model_evaluation(model=model, test_data=cancer_data_test, predict_target=cancer_target_test)
    print_classification_report(model=model, test_data=cancer_data_test, test_target=cancer_target_test)
    print('====================================================================================================')

======== VotingClassifier(estimators=[('svm', SVC()), ('dt', DecisionTreeClassifier()),
                             ('lr', LogisticRegression())]) Model Training ========
Model Evaluation:0.9649122807017544

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.91      0.95        43
           1       0.95      1.00      0.97        71

    accuracy                           0.96       114
   macro avg       0.97      0.95      0.96       114
weighted avg       0.97      0.96      0.96       114


======== StackingClassifier(estimators=[('svm', SVC()),
                               ('ada', DecisionTreeClassifier()),
                               ('lr', LogisticRegression())],
                   final_estimator=DecisionTreeClassifier()) Model Training ========
Model Evaluation:0.9824561403508771

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.98        43
  